# Introduction

This project focuses on the construction, optimisation, and risk management of a multi-asset equity portfolio - direct application of my modules in <b>DSC3705 - Financial Risk Management</b> & <b>INV3703 - Derivatives</b>. The selection of equities is intentionally based on market prominence and liquidity rather than prior fundamental or valuation analysis. As a result, expected returns are derived purely from historical ($\text 2025$) data and may be negative over the sampled period. This choice allows the project to isolate and evaluate the behaviour of portfolio optimisation, risk exposure, and hedging strategies independently of security selection or alpha generation, which is addressed separately in a dedicated fundamental analysis project.

# Importation of Libraries & Data.

<h2>Import necessary libraries.

In [63]:
!pip install yfinance --upgrade

In [64]:
import csv
import yfinance as yf
import pandas as pd
import warnings
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from scipy.stats import norm
from scipy.stats import linregress
from scipy.optimize import minimize

<h2>Import Data

Equities, imported with yfinance.

In [39]:
equities = [
    "NPN.JO",  # Naspers (Tech)
    "MTM.JO",  # Momentum (Finance)
    "AGL.JO",  # Anglo American (Mining)
    "FSR.JO",  # FirstRand (Banking)
    "SHP.JO",  # Shoprite (Retail)
    "MTN.JO",  # MTN Group (Telecom)
    "CPI.JO",  # Capitec (Finance)
    "SOL.JO",  # Sasol (Energy)
    "GFI.JO",  # Gold Fields (Gold)
    "CLS.JO"   # Clicks Group (Healthcare)
]
closing_prices = yf.download(equities, start="2025-01-01", end="2026-01-01")['Close']
eq_raw_returns = closing_prices.pct_change().dropna()

[*********************100%***********************]  10 of 10 completed


In [40]:
fig = px.line(eq_raw_returns, title='Daily Percentage Returns for All Equities (Interactive)')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Percentage Return')
fig.show()

Risk-free bonds: Government Bond 10-year bond data, sourced from <a href="https://za.investing.com/rates-bonds/south-africa-10-year-bond-yield-historical-data">za.investing.com</a>. The date range is 1 January 2025 to 1 January 2026. We include the <a href="https://za.investing.com/indices/ftse-jse-all-share">JSE All-Share Index</a> for benchmarking & comparison.

In [41]:
gov_data_r = pd.read_csv('/content/drive/MyDrive/South Africa 10-Year Bond Yield Historical Data.csv')
gov_data_r = gov_data_r[::-1].reset_index(drop=True)
gov_data_r.drop(gov_data_r.index[0], inplace=True)
rf = (gov_data_r['Price'].mean())/100
print(rf)

jasi = pd.read_csv('/content/drive/MyDrive/FTSE_JSE All Share Historical Data.csv')
jasi.drop(jasi.index[0], inplace=True)
jasi_returns = jasi['Change %'].dropna()

0.0969902409638554


9.699% is the average risk free rate over the past year.

The South African 10-year government bond yield is used as a proxy for the risk-free rate. While not truly risk-free, it reflects the opportunity cost faced by domestic investors.

# Optimising The Portfolio: Choosing the most efficient 5-share portfolio.


<b>Firstly,</b> we calculation of the historic <u>mean returns</u>, <u>the corresponding standard deviation</u>, and <u>covariance matrix</u>.

In [42]:
return_rates = eq_raw_returns.mean()
std_devs = eq_raw_returns.std()
cov_matrix = eq_raw_returns.cov()

<b>Secondly,</b> we calculate the <b>Sharpe Ratio </b> $ = \frac{\bar r_p - r_f}{\sigma_p}$ - a security analysis tool that quantifies how well a security performs compared to a risk-free security, the higher - the better.


We apply this method to choose 5 of the selected tickers with highest ratio to be part of the multi-asset portfolio.

In [43]:
warnings.filterwarnings('ignore', category=FutureWarning)

SR1 = (return_rates[0] - rf) / std_devs[0]
SR2 = (return_rates[1] - rf) / std_devs[1]
SR3 = (return_rates[2] - rf) / std_devs[2]
SR4 = (return_rates[3] - rf) / std_devs[3]
SR5 = (return_rates[4] - rf) / std_devs[4]
SR6 = (return_rates[5] - rf) / std_devs[5]
SR7 = (return_rates[6] - rf) / std_devs[6]
SR8 = (return_rates[7] - rf) / std_devs[7]
SR9 = (return_rates[8] - rf) / std_devs[8]
SR10 = (return_rates[9] - rf) / std_devs[9]

sharpe_ratios = {
    equities[0]: SR1,
    equities[1]: SR2,
    equities[2]: SR3,
    equities[3]: SR4,
    equities[4]: SR5,
    equities[5]: SR6,
    equities[6]: SR7,
    equities[7]: SR8,
    equities[8]: SR9,
    equities[9]: SR10
}

sharpe_series = pd.Series(sharpe_ratios)
sorted_sharpe_ratios = sharpe_series.sort_values(ascending=False)


print(sorted_sharpe_ratios.head())

CLS.JO   -2.577445
SHP.JO   -2.935343
CPI.JO   -4.173732
NPN.JO   -4.523444
SOL.JO   -4.863186
dtype: float64


So <u>Clicks ($SR_{10}$)</u>, <u>Shoprite ($SR_5$)</u>, <u>Capitec ($SR_7$)</u>, <u>Naspers ($SR_1$)</u>, and <u>Sasol ($SR_8$)</u> are the top 5 Sharpe ratios of the past year.
<br>
<br>
<br>
Here is how they perform.

In [44]:
portfolio_tickers = sorted_sharpe_ratios.index.tolist()[:5]
portfolio_closing_prices = closing_prices[portfolio_tickers]

df_long = portfolio_closing_prices.reset_index().melt(
    id_vars='Date', var_name='Ticker', value_name='Closing Price'
)

fig_share_prices = px.line(
    df_long,
    x='Date',
    y='Closing Price',
    color='Ticker',
    title='Daily Closing Prices (cents)',
    labels={'Closing Price': 'Closing Price', 'Date': 'Date'}
)

for trace in fig_share_prices.data:
    if "Trend" not in trace.name:
        trace.showlegend = False

for ticker in df_long['Ticker'].unique():
    df_ticker = df_long[df_long['Ticker'] == ticker]

    x_numeric = pd.to_numeric(df_ticker['Date'])
    y_values = df_ticker['Closing Price']

    slope, intercept, r_value, p_value, std_err = linregress(x_numeric, y_values)

    if slope > 0:
        trend_direction = "Positive Trend"
    elif slope < 0:
        trend_direction = "Negative Trend"
    else:
        trend_direction = "No Trend"

    trend_y = slope * x_numeric + intercept

    fig_share_prices.add_trace(
        go.Scatter(
            x=df_ticker['Date'],
            y=trend_y,
            mode='lines',
            name=f'{ticker} Trend ({trend_direction}, Slope: {slope:.2e})',
            line=dict(dash='dash'),
            showlegend=True
        )
    )

fig_share_prices.update_layout(hovermode="x unified")
fig_share_prices.show()

<b>Thirdly,</b> we use the <b>Markowitz Model</b> to determine the optimal weights, $w_i$, for each share, with the risk-free rate, $rf = 9.699%$%, as the benchmark
<br>
<br>
<b><u>The formulation</u></b><br>
To allow for short-selling, we do not impose non-negativity constraints on $w_i$. The objective is to minimise portfolio variance subject to the target return and the budget constraint:
\begin{aligned}
& {\text{minimise}} &
& \frac{1}{2} \sum_{i,j=1}^{5} w_i w_j \sigma_{ij} \\
& \text{subject to} & & \sum_{i=1}^{5} w_i \bar r_i >= 0.09699\\
& & & \sum_{i=1}^{5} w_i = 1
\end{aligned}
<br>
<br>
We can solve this problem using Lagrangian multipliers $\lambda$ and $\mu$. We form the Lagrangian: $\mathcal{L} = \frac{1}{2} \sum_{i,j=1}^{n} w_i w_j \sigma_{ij} - \lambda \left( \sum_{i=1}^{n} w_i \bar r_i - 0.09699 \right) - \mu \left( \sum_{i=1}^{n} w_i - 1 \right)$
<br>
<br>
The formula to be differentiated for 5 assets is: \begin{aligned}
\mathcal{L} &= \frac{1}{2} \left( \sum_{i=1}^{5} w_i^2 \sigma_i^2 + 2 \sum_{i,j=1}^{5} w_i w_j \sigma_{ij} \right) - \lambda (w_1\bar r_1 + w_2\bar r_2 + w_3\bar r_3 + w_4\bar r_4 + w_5\bar r_5 - 0.09699) - \mu (w_1 + w_2 + w_3 + w_4 + w_5 - 1)
\end{aligned}
<br>
<br>
Hence,
\begin{aligned}
\frac{\partial \mathcal{L}}{\partial w_1} &= \frac{1}{2} (2\sigma_1^2 w_1 + 2\sigma_{12}w_2 + 2\sigma_{13}w_3 + 2\sigma_{14}w_4 + 2\sigma_{15}w_5) - \lambda \bar r_1 - \mu \\
&\vdots \\
\frac{\partial \mathcal{L}}{\partial w_5} &= \frac{1}{2} (2\sigma_5^2 w_5 + 2\sigma_{51}w_1 + 2\sigma_{52}w_2 + 2\sigma_{53}w_3 + 2\sigma_{54}w_4) - \lambda \bar r_5 - \mu
\end{aligned}
<br>
<br>
<br>

In [45]:
portfolio_returns = return_rates[portfolio_tickers]
portfolio_stds = std_devs[portfolio_tickers]
portfolio_cov_matrix = cov_matrix.loc[portfolio_tickers, portfolio_tickers]

print("Portfolio Mean Returns:\n", portfolio_returns)
print("\nPortfolio Standard Deviations:\n", portfolio_stds)
print("\nPortfolio Covariance Matrix:\n", portfolio_cov_matrix)

Portfolio Mean Returns:
 Ticker
CLS.JO   -0.000329
SHP.JO   -0.000257
CPI.JO    0.001269
NPN.JO    0.001378
SOL.JO    0.001421
dtype: float64

Portfolio Standard Deviations:
 Ticker
CLS.JO    0.012562
SHP.JO    0.014418
CPI.JO    0.015001
NPN.JO    0.019660
SOL.JO    0.037079
dtype: float64

Portfolio Covariance Matrix:
 Ticker    CLS.JO    SHP.JO    CPI.JO    NPN.JO    SOL.JO
Ticker                                                  
CLS.JO  0.000158  0.000099  0.000090  0.000022  0.000034
SHP.JO  0.000099  0.000208  0.000096  0.000019  0.000073
CPI.JO  0.000090  0.000096  0.000225  0.000072  0.000054
NPN.JO  0.000022  0.000019  0.000072  0.000387  0.000090
SOL.JO  0.000034  0.000073  0.000054  0.000090  0.001375


In [46]:
def portfolio_variance(weights, cov_matrix):
    return weights.T @ cov_matrix @ weights

def get_optimal_weights(mean_returns, cov_matrix, target_return):
    num_assets = len(mean_returns)


    epsilon = 1e-6

    constraints = [
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},

        {'type': 'ineq', 'fun': lambda x: np.sum(x * mean_returns) - target_return - epsilon}
    ]

    init_guess = num_assets * [1. / num_assets]

    optimal_result = minimize(
        portfolio_variance, init_guess, args=(cov_matrix,), method='SLSQP',
        bounds=None, constraints=constraints
    )
    if optimal_result.success:
        return optimal_result.x
    else:
        raise ValueError("Optimisation failed: " + optimal_result.message)

portfolio_returns_np = portfolio_returns.to_numpy()
portfolio_cov_matrix_np = portfolio_cov_matrix.to_numpy()

target_return = (1 + rf)**(1/252) - 1

optimal_weights = get_optimal_weights(portfolio_returns_np, portfolio_cov_matrix_np, target_return)

optimal_weights_series = pd.Series(optimal_weights, index=portfolio_tickers)

print("Optimal Weights for the Portfolio:\n", optimal_weights_series)

expected_portfolio_return = np.sum(optimal_weights_series * portfolio_returns)
expected_portfolio_volatility = np.sqrt(portfolio_variance(optimal_weights_series, portfolio_cov_matrix))

print(f"\nExpected Portfolio Return: {expected_portfolio_return:.6f}")
print(f"Expected Portfolio Volatility: {expected_portfolio_volatility:.6f}")

Optimal Weights for the Portfolio:
 CLS.JO    0.2
SHP.JO    0.2
CPI.JO    0.2
NPN.JO    0.2
SOL.JO    0.2
dtype: float64

Expected Portfolio Return: 0.000696
Expected Portfolio Volatility: 0.012088


The <b>daily</b> <u>expected portfolio return</u> is 0.0696% , which is <i>much</i> higher than the daily risk-free (target return) of 0.0367%. So the weights remain exactly as the initial guess (20% each - 5 shares) because the model prioritises variance minimisation over return differentiation. This result highlights the sensitivity of constrained optimisation to return targets and necessitates the subsequent implementation of the tangency portfolio, which explicitly maximises excess return per unit of risk.

In [47]:
portfolio_returns_np = portfolio_returns.to_numpy()
portfolio_cov_matrix_np = portfolio_cov_matrix.to_numpy()

def portfolio_variance(weights, cov_matrix):
    return weights.T @ cov_matrix @ weights

def get_min_variance_weights(mean_returns, cov_matrix, target_return):
    num_assets = len(mean_returns)
    constraints = [
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
        {'type': 'eq', 'fun': lambda x: np.sum(x * mean_returns) - target_return}
    ]
    init_guess = num_assets * [1. / num_assets]
    result = minimize(portfolio_variance, init_guess, args=(cov_matrix,),
                      method='SLSQP', bounds=None, constraints=constraints)
    return result.x if result.success else None

min_r = portfolio_returns_np.min()
max_r = portfolio_returns_np.max()
target_returns = np.linspace(min_r * 2, max_r * 2, 100)

results = []
for tr in target_returns:
    w = get_min_variance_weights(portfolio_returns_np, portfolio_cov_matrix_np, tr)
    if w is not None:
        vol = np.sqrt(portfolio_variance(w, portfolio_cov_matrix_np))
        shorted = [portfolio_tickers[i] for i, weight in enumerate(w) if weight < 0]
        results.append({
            'Volatility': vol,
            'Return': tr,
            'Shorted Assets': ", ".join(shorted) if shorted else "None"
        })

efficiency_frontier_df = pd.DataFrame(results)

fig = px.line(
    efficiency_frontier_df, x='Volatility', y='Return',
    title='Efficient Frontier (Short-Selling Allowed)',
    labels={'Volatility': 'Risk (Std Dev)', 'Return': 'Expected Daily Return'},
    hover_data=['Shorted Assets']
)

for i, ticker in enumerate(portfolio_tickers):
    fig.add_scatter(
        x=[np.sqrt(portfolio_cov_matrix_np[i,i])],
        y=[portfolio_returns_np[i]],
        mode='markers+text', name=ticker, text=[ticker], textposition="top center"
    )

fig.update_layout(template="plotly_white", hovermode="closest")
fig.show()

# Tangency Portfolio: Incorporation of the risk-free asset.


The applicaton of the One Fund Theorem: To maximise the Sharpe Ratio (Tangency Portfolio) weights:
 \begin{aligned}
\sigma_1^2 w_1 + \sigma_{12} w_2 + \dots + \sigma_{15} w_5 &= \bar r_1 - r_f \\
&\vdots \\
\sigma_5^2 w_5 + \sigma_{51} w_1 + \dots + \sigma_{54} w_4 &= \bar r_5 - r_f.
\end{aligned}
<br>

To find the weights, we solve the system of linear equations derived from the First Order Conditions:
\begin{equation}
    \Sigma \mathbf{z} = (\bar r - r_f \mathbf{1}) \implies \mathbf{z} = \Sigma^{-1}(\bar r - r_f \mathbf{1})
\end{equation}

The final weights are obtained by normalising $\mathbf{z} $  such that $ \sum w_i = 1:
\begin{equation}
    w_i = \frac{z_i}{\sum_{j=1}^N z_j}
\end{equation}
$
<br>
<br>


In [48]:
target_return = (1 + rf)**(1/252) - 1

mu_np = portfolio_returns.to_numpy()

sigma_inv = np.linalg.inv(portfolio_cov_matrix)

ones_vector = np.ones_like(mu_np)

excess_returns = mu_np - (target_return * ones_vector)

numerator = sigma_inv @ excess_returns

denominator = ones_vector.T @ numerator

w_tan = numerator / denominator

tangency_portfolio_weights = pd.Series(w_tan, index=portfolio_tickers)

print("Tangency Portfolio Weights:\n", tangency_portfolio_weights)

expected_tangency_return = np.sum(tangency_portfolio_weights * portfolio_returns)
expected_tangency_volatility = np.sqrt(tangency_portfolio_weights.T @ portfolio_cov_matrix.values @ tangency_portfolio_weights)

print(f"\nExpected Tangency Portfolio Return: {expected_tangency_return:.6f}")
print(f"Expected Tangency Portfolio Volatility: {expected_tangency_volatility:.6f}")

Tangency Portfolio Weights:
 CLS.JO    11.682911
SHP.JO     6.251105
CPI.JO   -13.004834
NPN.JO    -2.692715
SOL.JO    -1.236468
dtype: float64

Expected Tangency Portfolio Return: -0.027431
Expected Tangency Portfolio Volatility: 0.217221


The tangency portfolio exhibits massive positive and negative weights, reflecting the allowance of unrestricted short selling and leverage. Such extreme allocations are a known consequence of mean–variance optimisation when expected returns are estimated from historical data, as small estimation errors in mean returns are amplified through the inversion of the covariance matrix. So instead of representing a practical, implementable portfolio, the tangency portfolio serves as caution, showing the theoretical implications and limitations of classical portfolio optimisation.

In [49]:
tangency_return = expected_tangency_return
tangency_volatility = expected_tangency_volatility

fig = px.line(
    efficiency_frontier_df,
    x='Volatility',
    y='Return',
    title='Efficiency Frontier for the 5 Equities with Tangency Portfolio',
    labels={'Volatility': 'Daily Volatility (Standard Deviation)', 'Return': 'Daily Expected Return'}
)

fig.add_scatter(
    x=[tangency_volatility],
    y=[tangency_return],
    mode='markers',
    marker=dict(size=10, color='red', symbol='star'),
    name='Tangency Portfolio',
    hoverinfo='text',
    text=[
        f'Tangency Portfolio<br>Return: {tangency_return:.6f}<br>Volatility: {tangency_volatility:.6f}'
    ]
)

fig.update_layout(hovermode="x unified")
fig.show()

The tangency portfolio weights are calculated as follows:
*   `CLS.JO`: 11.682887
*   `SHP.JO`: 6.251109
*   `CPI.JO`: -13.004818
*   `NPN.JO`: -2.692711
*   `SOL.JO`: -1.236467

The portfolio structuring involves substantial short positions on `CPI.JO`, `NPN.JO`, and `SOL.JO`, with highly leveraged long positions in `CLS.JO` and `SHP.JO`. This suggests a highly leveraged strategy by the model, to maximise the Sharpe ratio, often indicating that the assets being shorted have lower historic and/or expected returns or higher correlations that make them undesired in a risk-efficient portfolio. This is a direct consequence of my decision to choose the shares solely based on popularity, not on sound analysis. <br><br>The resulting portfolio has an <u><b>expected return</b></u> of <u>-0.027431</u> and a <u><b>volatility</b></u> of <u>0.217221</u>, yielding a $ \text{coefficient of variation = } \frac{\sigma_p}{\mu_p} = \frac{0.217221}{-0.027431} = -7.9188 ≈ -7.92$.
<br>
<h3> Model Solution</h3>:
*   The tangency portfolio weights were successfully calculated:
    *   `CLS.JO`: 11.682887
    *   `SHP.JO`: 6.251109
    *   `CPI.JO`: -13.004818
    *   `NPN.JO`: -2.692711
    *   `SOL.JO`: -1.236467
*   The expected return for the calculated tangency portfolio is -0.027431 - a projected daily loss.
*   The expected volatility for the calculated tangency portfolio is 0.217221 - relatively volatile/unstable.

# Sensitivity Analysis: Impact of Risk-Free Rate on Tangency Portfolio



Sensitivity analysis is crucial to understand the resilience of our tangency portfolio. In this section, I explore how variation in the risk-free rate ($r_f$), impacts the weights, expected return, sharpe ratio, and volatility of the tangency portfolio.

In [50]:
rf_values = np.linspace(0.00, 0.21, 100)
sensitivity_results = []

for current_rf in rf_values:
    daily_rf_rate = (1 + current_rf)**(1/252) - 1

    excess_returns_rf = mu_np - (daily_rf_rate * ones_vector)

    numerator_rf = sigma_inv @ excess_returns_rf
    denominator_rf = ones_vector.T @ numerator_rf

    if np.abs(denominator_rf) < 1e-9:
        w_tan_rf = np.zeros_like(mu_np)
        expected_tangency_return_rf = np.nan
        expected_tangency_volatility_rf = np.nan
        sharpe_ratio_rf = np.nan
    else:
        w_tan_rf = numerator_rf / denominator_rf

        expected_tangency_return_rf = np.sum(w_tan_rf * portfolio_returns_np)
        expected_tangency_volatility_rf = np.sqrt(w_tan_rf.T @ portfolio_cov_matrix_np @ w_tan_rf)

        if expected_tangency_volatility_rf > 0: # Avoid division by zero
            sharpe_ratio_rf = (expected_tangency_return_rf - daily_rf_rate) / expected_tangency_volatility_rf
        else:
            sharpe_ratio_rf = np.nan

    sensitivity_results.append({
        'Risk_Free_Rate': current_rf,
        'Weights': list(w_tan_rf),
        'Expected_Return': expected_tangency_return_rf,
        'Expected_Volatility': expected_tangency_volatility_rf,
        'Sharpe_Ratio': sharpe_ratio_rf
    })


rf_sensitivity_df = pd.DataFrame(sensitivity_results)

display(rf_sensitivity_df.head())

,Risk_Free_Rate,Weights,Expected_Return,Expected_Volatility,Sharpe_Ratio
0,0.000000,"[-1.849021426449644, -1.0031381139829099, 2.77...",0.005886,0.044836,0.131287
1,0.002121,"[-1.9130921301035309, -1.037485350345211, 2.84...",0.006044,0.046038,0.131102
2,0.004242,"[-1.9807435683813979, -1.0737521588593166, 2.9...",0.006211,0.047310,0.130922
3,0.006364,"[-2.052284710989816, -1.1121041730353205, 3.01...",0.006387,0.048656,0.130747
4,0.008485,"[-2.128061126202011, -1.1527266462649124, 3.10...",0.006573,0.050084,0.130578


In [51]:
fig_weights = go.Figure()

num_assets = len(portfolio_tickers)
for i, ticker in enumerate(portfolio_tickers):
    weights_for_ticker = [weights[i] for weights in rf_sensitivity_df['Weights']]
    fig_weights.add_trace(go.Scatter(
        x=rf_sensitivity_df['Risk_Free_Rate'],
        y=weights_for_ticker,
        mode='lines',
        name=f'Weight for {ticker}'
    ))

fig_weights.update_layout(
    title='Portfolio Weights to Risk-Free Rate',
    xaxis_title='Risk-Free Rate (Annual)',
    yaxis_title='Asset Weight',
    hovermode='x unified'
)
fig_weights.show()


In [52]:
fig_return = px.line(
    rf_sensitivity_df,
    x='Risk_Free_Rate',
    y='Expected_Return',
    title='Expected Return to Risk-Free Rate',
    labels={
        'Risk_Free_Rate': 'Risk-Free Rate (Annual)',
        'Expected_Return': 'Expected Portfolio Return'
    }
)
fig_return.update_layout(hovermode='x unified')
fig_return.show()

In [53]:
fig_sharpe = px.line(
    rf_sensitivity_df,
    x='Risk_Free_Rate',
    y='Sharpe_Ratio',
    title='Sharpe Ratio to Risk-Free Rate',
    labels={
        'Risk_Free_Rate': 'Risk-Free Rate (Annual)',
        'Sharpe_Ratio': 'Sharpe Ratio'
    }
)
fig_sharpe.update_layout(hovermode='x unified')
fig_sharpe.show()

In [54]:
fig_volatility = px.line(
    rf_sensitivity_df,
    x='Risk_Free_Rate',
    y='Expected_Volatility',
    title='Expected Volatility to Risk-Free Rate',
    labels={
        'Risk_Free_Rate': 'Risk-Free Rate (Annual)',
        'Expected_Volatility': 'Expected Portfolio Volatility'
    }
)
fig_volatility.update_layout(hovermode='x unified')
fig_volatility.show()

<h3>Overview:</h3>


Sharpe Ratio for tangency portfolios we calculated for a range of risk-free rates, from $0$% to $15$%, along with weights, expected return, and expected volatility for each tangency portfolio at different risk-free rate levels.

<b>Expected Return:</b> It appears that $\approx 7.9$% is the point of inflection for the tangency portfolio <u>expected return</u> - the return reaches its <u>peak</u> ($\approx 28.5$%) at $r_f=7.8$%, and thereafter drops to its <u>trough</u> ($\approx -94.56$%) at $r_f=8.06$%.

<b>Portfolio Weights:</b> There is also an inflection at the same points - the weights a flipped from long to short - short to long. At this point $μ - r_f$ has become so negative that the optimal tangency portfolio is short these assets.

<b>The Sharpe Ratio</b>: positive ratio ($0.127$) at $r_f = 7.8$%, to a negative($-0.127$) ratio at $r_f=8.06$%.

<b>Volatility</b>: Moves moderately, then gradually increases, and then exponentially reaches its peak ($740$%) at $r_f=8.06$%, thereafter gradually returns to its average of $3.78$%. This demonstrates the mathematical instability of unconstrained mean-variance optimisation when excess returns are near zero, a well-known issue in portfolio theory.

# Risk Management: Incorporating derivatives.

<h3></h3>Since this porftolio is projected to lose money, to leverage/hedge against projected losses, a <b>mixed-strategy</b> is perfect because of the long and short positions in the portfolio.
<br>
<br>
The model's result is to $\textit {long} $ Clicks ($w \approx $ 1,168.2%) and Shoprite ($w \approx $ 625.11%) and then $\textit {short}$ Capitec ($w \approx $ 1,300.5%), Naspers ($w \approx $ 269.27%) , and Sasol ($w \approx $ 123.65%).
These weights, all of them, are massive - <i>highly</i> leveraged.
<br>

<h3><u>Long positions strategy:</u></h3>
<b>Clicks and Shoprite</b>'s expected returns are negative. The model, however, suggest we take massive long positions on the two. So the best strategy is to <b>purchase</b> a $\textit {protective put} $ - which protects us againts price decline - it grants us the right to sell the shares at a particular strike price.

<br>
<br>
<h3><u>Short positions strategy:</u></h3>
<b>Capitec, Naspers, and Sasol</b> also have negative expected returns. The model suggests we take massive short positions on these three. The strategy to deploy is to <b>purchase</b> a $\textit {protective call} $ - which protects us from price appreciation <i>(short squeeze)</i> - it grants us the right to buy the shares at a particular strike price.
<br>
<br>
<h3><u>Simulation</u></h3>

The 2025 share data used to construct the tangency portfolio is crucial input to the simulation of the share prices.

<u>Method:</u><br>
<b>First:</b> Use Geometric Brownian Motion to generate 10,000 paths:

$$ S_t = S_{t-1} \cdot \exp\left[ \left( \mu - \frac{\sigma^2}{2} \right) dt + \sigma \sqrt{dt} \cdot Z \right] $$

$$
\begin{aligned}
\mu &: \text{Share expected return } \\
\sigma &: \text{Volatility } \\
Z &: \text{Critical value, } Z \sim \mathcal{N}(0,1)
\end{aligned}
$$
<br>
<b>Secondly:</b> Value the puts and calls payoffs with strike prices $X$ set at $X = S_{0,i}$ per $i^{th}$ share - the $\text{At-The-Money}$ strategy.
<br>
<br>
<h3><u> Long Position Shares.</u></h3>

In practice, it is desirable to choose options with high delta to ensure high degree of protection because delta measures sentivity and risk - the rate of change in an option's value with respect to the rate of change in the underlying's price: $\text{Delta} (\Delta) = \frac{\partial V}{\partial S}$. However, due to limited information on options data - delta would be avoided for now and apply static hedging strategy.<br>
<br>
The data available for premium pricing is the 2025 volatilities and the risk-free rate. These variables: $d_1 = \frac{\ln(S_0 / K) + (r + \frac{\sigma^2}{2})T}{\sigma \sqrt{T}}$ and $d_2 = d_1 - \sigma \sqrt{T}$ are the inputs to these pricing equations $$\text{Call Premium } (C) = S_0 N(d_1) - Ke^{-rT} N(d_2)$$

$$\text{Put Premium } (P) = Ke^{-rT} N(-d_2) - S_0 N(-d_1)$$
<br>
<br>

<h3><u>Share prices at start of 2026.</u></h3>
$\text{Clicks } S_0 = X_{cls}= 331.94 - long$<br>
$\text{Shoprite } S_0 = X_{shp} = 271.91 - long$<br>
$\text{Capitec } S_0 = X_{cpi} = 4189.5 - short$<br>
$\text{Naspers } S_0 = X_{npn} = 13.55(USD) ≈ 13.55 × 16.47 = 223.17(ZAR)- short$<br>
$\text{Sasol } S_0 = X_{sos} = 104.76 - short$

In [55]:
share_params = {
    'CLS.JO': {'S0': 331.94, 'X': 331.94},
    'SHP.JO': {'S0': 271.91, 'X': 271.91},
    'CPI.JO': {'S0': 4189.5, 'X': 4189.5},
    'NPN.JO': {'S0': 223.17, 'X': 223.17},
    'SOL.JO': {'S0': 104.76, 'X': 104.76}
}

trading_days = 252
annual_volatilities = portfolio_stds * np.sqrt(trading_days)

T = 1
print(f"Time to maturity (T) set to: {T} year")

def d1(S0, X, T, rf, sigma):
    return (np.log(S0 / X) + (rf + sigma**2 / 2) * T) / (sigma * np.sqrt(T))

def d2(S0, X, T, rf, sigma):
    return d1(S0, X, T, rf, sigma) - sigma * np.sqrt(T)

def call_premium(S0, X, T, rf, sigma):
    d_1 = d1(S0, X, T, rf, sigma)
    d_2 = d2(S0, X, T, rf, sigma)
    return S0 * norm.cdf(d_1) - X * np.exp(-rf * T) * norm.cdf(d_2)

def put_premium(S0, X, T, rf, sigma):
    d_1 = d1(S0, X, T, rf, sigma)
    d_2 = d2(S0, X, T, rf, sigma)
    return X * np.exp(-rf * T) * norm.cdf(-d_2) - S0 * norm.cdf(-d_1)
option_premiums = {}

long_position_tickers = ['CLS.JO', 'SHP.JO']
short_position_tickers = ['CPI.JO', 'NPN.JO', 'SOL.JO']

for ticker in portfolio_tickers:
    S0 = share_params[ticker]['S0']
    X = share_params[ticker]['X']
    sigma = annual_volatilities[ticker]

    if ticker in long_position_tickers:
        premium = put_premium(S0, X, T, rf, sigma)
        option_premiums[ticker] = {'Type': 'Put', 'Premium': premium}
    elif ticker in short_position_tickers:
        premium = call_premium(S0, X, T, rf, sigma)
        option_premiums[ticker] = {'Type': 'Call', 'Premium': premium}

print("Option Premiums:")
for ticker, details in option_premiums.items():
    print(f"  {ticker}: {details['Type']} Premium = {details['Premium']:.4f}")

Time to maturity (T) set to: 1 year
Option Premiums:
  CLS.JO: Put Premium = 12.7077
  SHP.JO: Put Premium = 13.1415
  CPI.JO: Call Premium = 603.1554
  NPN.JO: Call Premium = 37.9627
  SOL.JO: Call Premium = 28.2744


<h3><u>Option Premiums Implications to Risk</u></h3>

The calculated option premiums provide critical insights for the proposed risk management strategy:

<b>Long Positions (CLS.JO and SHP.JO - Protective Puts):</b>
*   **CLS.JO Put Premium:** 12.7077
*   **SHP.JO Put Premium:** 13.1415

The costs, 12.71 and 13.14 respectively, are the cost incurred - per share - to secure this downside protection over the one-year maturity period.

<b>Short Positions (CPI.JO, NPN.JO, and SOL.JO - Protective Calls):</b>
*   **CPI.JO Call Premium:** 603.1554
*   **NPN.JO Call Premium:** 37.9627
*   **SOL.JO Call Premium:** 28.2744

The costs are significantly higher for Capitec (603.16) compared to Naspers (37.96) and Sasol (28.27). This difference reflects Capitec's much higher share price (4189.5) and potentially its volatility relative to its price, making it more expensive to hedge against upward movements.

<h3><u>Overall Implications:</u></h3>

The cost of implementing this hedging strategy would be the sum of these premiums. Given the highly leveraged nature of the tangency portfolio, these hedging costs are crucial for managing the risks associated with such massive long and short positions.
<br>



<h3><u>Implement Geometric Brownian Motion Simulation</u></h3>


In [56]:
initial_prices_S0 = {
    'CLS.JO': 331.94,
    'SHP.JO': 271.91,
    'CPI.JO': 4189.5,
    'NPN.JO': 223.17,
    'SOL.JO': 104.76
}

num_trading_days = 252
num_simulations = 10000

mu_daily = portfolio_returns[portfolio_tickers]

sigma_daily = portfolio_stds[portfolio_tickers]

def simulate_gbm_paths(S0, mu, sigma, num_trading_days, num_simulations):

    price_paths = np.zeros((num_trading_days + 1, num_simulations))
    price_paths[0] = S0

    dt = 1
    for t in range(1, num_trading_days + 1):
        Z = np.random.standard_normal(num_simulations)
        price_paths[t] = price_paths[t-1] * np.exp((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)

    return price_paths

simulated_paths = {}

for ticker in portfolio_tickers:
    S0 = initial_prices_S0[ticker]
    mu = mu_daily[ticker]
    sigma = sigma_daily[ticker]


    paths = simulate_gbm_paths(S0, mu, sigma, num_trading_days, num_simulations)
    simulated_paths[ticker] = paths


In [57]:
fig = go.Figure()

num_sample_paths = 50

for ticker in portfolio_tickers:
    ticker_paths = simulated_paths[ticker]

    for i in range(num_sample_paths):
        fig.add_trace(
            go.Scatter(
                x=np.arange(num_trading_days + 1),
                y=ticker_paths[:, i],
                mode='lines',
                line=dict(width=1),
                name=f'{ticker} Path {i+1}',
                showlegend=False,
                line_color='rgba(0,0,0,0.1)' if ticker == 'CLS.JO' else
                             'rgba(0,0,255,0.1)' if ticker == 'SHP.JO' else
                             'rgba(255,0,0,0.1)' if ticker == 'CPI.JO' else
                             'rgba(0,255,0,0.1)' if ticker == 'NPN.JO' else
                             'rgba(255,255,0,0.1)'
            )
        )


    mean_path = ticker_paths.mean(axis=1)
    fig.add_trace(
        go.Scatter(
            x=np.arange(num_trading_days + 1),
            y=mean_path,
            mode='lines',
            line=dict(width=2),
            name=f'{ticker} Mean Path',
            showlegend=True,
            line_color='black' if ticker == 'CLS.JO' else
                         'blue' if ticker == 'SHP.JO' else
                         'red' if ticker == 'CPI.JO' else
                         'green' if ticker == 'NPN.JO' else
                         'yellow'
        )
    )


fig.update_layout(
    title='Geometric Brownian Motion Simulation of Portfolio Share Prices (2026)',
    xaxis_title='Trading Day',
    yaxis_title='Share Price',
    hovermode='x unified'
)


fig.show()

<h3><u>Simulation Insights</u></h3>


The simulated share prices for the five portfolio tickers indicate a range of potential outcomes for each. <b>CLS.JO, SHP.JO, NPN.JO, and SOL.JO</b> show <b><i>downward</b></i> trends in their paths over the 252 trading days, suggesting a potential value is being destroyed.

Conversely, <b>CPI.JO</b> exhibits an upward trending path, indicating an <b>value creation</b>. The individual simulation paths for each ticker spread out over time, illustrating increasing uncertainty and a wider potential price range further into the future. This suggests that while CPI.JO might be a strong performer, the other tickers could drag down overall portfolio value.

<b>Model Parameters</b><br> The simulation used initial share prices ranging from 104.76 (SOL.JO) to 4189.5 (CPI.JO). Daily expected returns ($\mu$) were very small, ranging from approximately -0.000329 (CLS.JO) to 0.000085 (CPI.JO), while daily volatilities ($\sigma$) ranged from 0.012562 (CLS.JO) to 0.021021 (SOL.JO). A total of 10,000 simulations were run over 252 trading days.
*   **Simulation Execution**: A Geometric Brownian Motion simulation function was successfully implemented and used to generate 10,000 price paths for each of the five shares over 252 trading days.
*   **Share Price Trends**:
    *   **CPI.JO** shows an overall upward trend in its mean simulated price path, indicating potential growth. Its price uncertainty also grows significantly over the simulation period.
    *   **CLS.JO, SHP.JO, NPN.JO, and SOL.JO** generally exhibit downward trends in their mean simulated price paths, suggesting a potential decline in value over the 252 trading days. The price uncertainty for these shares also expands over time.
*   **Price Volatility and Range**: For all shares, the spread of simulated prices widens considerably as the number of trading days increases, highlighting the growing uncertainty in price predictions over longer horizons - as is the case for most securities.


<h3><u>Calculation of the Options payoffs</u></h3>

For simplicity purposes, the option are to be potentially exercised at maturity, a year later - end of 2026.

In [58]:
def call_payoff(St, X):
    return np.maximum(0, St - X)

def put_payoff(St, X):
    return np.maximum(0, X - St)

individual_option_payoffs = {}

for ticker in portfolio_tickers:
    X = initial_prices_S0[ticker]

    St = simulated_paths[ticker][-1, :]

    if ticker in long_position_tickers:
        payoffs = put_payoff(St, X)
        individual_option_payoffs[ticker] = {'Type': 'Put', 'Payoffs': payoffs}
    elif ticker in short_position_tickers:
        payoffs = call_payoff(St, X)
        individual_option_payoffs[ticker] = {'Type': 'Call', 'Payoffs': payoffs}

option_summary_data = []

for ticker in portfolio_tickers:
    option_type = individual_option_payoffs[ticker]['Type']
    premium = option_premiums[ticker]['Premium']
    average_payoff = np.mean(individual_option_payoffs[ticker]['Payoffs'])
    net_profit = average_payoff - premium

    option_summary_data.append({
        'Ticker': ticker,
        '': option_type,
        'Average Payoff - ': average_payoff,
        'Initial Premium': premium,
        ' = Net Gain/Loss': net_profit
    })

option_summary_df = pd.DataFrame(option_summary_data)
option_summary_df = option_summary_df.set_index('Ticker')
display(option_summary_df)

,,Average Payoff -,Initial Premium,= Net Gain/Loss
Ticker,,,,
CLS.JO,Put,41.164859,12.707684,28.457174
SHP.JO,Put,33.878138,13.141459,20.736679
CPI.JO,Call,1637.102414,603.155448,1033.946965
NPN.JO,Call,98.548615,37.962726,60.585889
SOL.JO,Call,56.995685,28.274364,28.721320


<h3>Analysing Individual Share Contributions to Portfolio Performance</h3>

This is done by multiplying each stock's tangency weight by its average final simulated price ($S_t$) across all runs.

In [59]:
individual_contributions = {}
unhedged_portfolio_final_values = np.zeros(num_simulations)

for i in range(num_simulations):
    final_sim_prices = np.array([simulated_paths[ticker][-1, i] for ticker in portfolio_tickers])

    unhedged_portfolio_final_values[i] = np.sum(tangency_portfolio_weights * final_sim_prices)


average_final_sim_prices = {ticker: np.mean(simulated_paths[ticker][-1, :]) for ticker in portfolio_tickers}

print("Average Simulated Last Prices per Share:")
for ticker, price in average_final_sim_prices.items():
    print(f"  {ticker}: {price:.2f}")

print("\nTangency Portfolio Weights:")
print(tangency_portfolio_weights)

for ticker in portfolio_tickers:
    weight = tangency_portfolio_weights[ticker]
    avg_final_price = average_final_sim_prices[ticker]
    contribution = weight * avg_final_price
    individual_contributions[ticker] = contribution

contributions_series = pd.Series(individual_contributions)

print("\nAverage Monetary Contribution of Each Stock to the Final Unhedged Portfolio Value:")
display(contributions_series.sort_values(ascending=True))

print(f"\nSum of individual contributions: {contributions_series.sum():.4f}")
print(f"Mean unhedged portfolio value: {np.mean(unhedged_portfolio_final_values):.4f}")

Average Simulated Last Prices per Share:
  CLS.JO: 304.89
  SHP.JO: 254.65
  CPI.JO: 5778.33
  NPN.JO: 316.24
  SOL.JO: 149.69

Tangency Portfolio Weights:
CLS.JO    11.682911
SHP.JO     6.251105
CPI.JO   -13.004834
NPN.JO    -2.692715
SOL.JO    -1.236468
dtype: float64

Average Monetary Contribution of Each Stock to the Final Unhedged Portfolio Value:


,0
CPI.JO,-75146.205022
NPN.JO,-851.547510
SOL.JO,-185.088935
SHP.JO,1591.856005
CLS.JO,3561.946406



Sum of individual contributions: -71029.0391
Mean unhedged portfolio value: -71029.0391


The shorted shares' prices did not decline enough to make a positive contribution to the portfolio, and Capited with its massive weight has negatively contributed to the portfolio metrics.

A highly leveraged investment with negative projected returns is financially insensible. However, this project is purely to apply my theoretical knowledge and have a more indepth understand of my course material to construct a portfolio - and manage the risks involved with derivatives.

Below we also include Value at Risk calculations

<h3><u>Hedged vs Unhedged Portfolio</u></h3>

In [60]:
hedged_portfolio_final_values = np.zeros(num_simulations)

total_option_premium = sum(details['Premium'] for details in option_premiums.values())

for i in range(num_simulations):
    final_sim_prices = np.array([simulated_paths[ticker][-1, i] for ticker in portfolio_tickers])

    portfolio_value_underlying = np.sum(tangency_portfolio_weights * final_sim_prices)

    total_option_payoff = 0
    for ticker in portfolio_tickers:
        total_option_payoff += individual_option_payoffs[ticker]['Payoffs'][i]

    hedged_portfolio_final_values[i] = portfolio_value_underlying + total_option_payoff - total_option_premium


mean_unhedged = np.mean(unhedged_portfolio_final_values)
median_unhedged = np.median(unhedged_portfolio_final_values)
std_unhedged = np.std(unhedged_portfolio_final_values)
min_unhedged = np.min(unhedged_portfolio_final_values)
max_unhedged = np.max(unhedged_portfolio_final_values)
range_unhedged = max_unhedged - min_unhedged
var_unhedged_5_percent = np.percentile(unhedged_portfolio_final_values, 5)
es_unhedged_5_percent = np.mean(unhedged_portfolio_final_values[unhedged_portfolio_final_values <= var_unhedged_5_percent])

mean_hedged = np.mean(hedged_portfolio_final_values)
median_hedged = np.median(hedged_portfolio_final_values)
std_hedged = np.std(hedged_portfolio_final_values)
min_hedged = np.min(hedged_portfolio_final_values)
max_hedged = np.max(hedged_portfolio_final_values)
range_hedged = max_hedged - min_hedged
var_hedged_5_percent = np.percentile(hedged_portfolio_final_values, 5)
es_hedged_5_percent = np.mean(hedged_portfolio_final_values[hedged_portfolio_final_values <= var_hedged_5_percent])

print("--- Portfolio Final Value Statistics ---")
print("\nUnhedged Portfolio:")
print(f"  Mean:             {mean_unhedged:,.2f}")
print(f"  Median:           {median_unhedged:,.2f}")
print(f"  Std Deviation:    {std_unhedged:,.2f}")
print(f"  Range:            {range_unhedged:,.2f}")
print(f"  Min:              {min_unhedged:,.2f}")
print(f"  Max:              {max_unhedged:,.2f}")
print(f"  VaR (5%):         {var_unhedged_5_percent:,.2f}")
print(f"  Expected SF(5%):  {es_unhedged_5_percent:,.2f}")

print("\nHedged Portfolio:")
print(f"  Mean:             {mean_hedged:,.2f}")
print(f"  Median:           {median_hedged:,.2f}")
print(f"  Std Deviation:    {std_hedged:,.2f}")
print(f"  Range:            {range_hedged:,.2f}")
print(f"  Min:              {min_hedged:,.2f}")
print(f"  Max:              {max_hedged:,.2f}")
print(f"  VaR (5%):         {var_hedged_5_percent:,.2f}")
print(f"  Expected SF (5%): {es_hedged_5_percent:,.2f}")

--- Portfolio Final Value Statistics ---

Unhedged Portfolio:
  Mean:             -71,029.04
  Median:           -68,804.65
  Std Deviation:    18,287.11
  Range:            173,274.99
  Min:              -194,707.14
  Max:              -21,432.15
  VaR (5%):         -103,985.56
  Expected SF(5%):  -116,244.51

Hedged Portfolio:
  Mean:             -69,856.59
  Median:           -67,887.37
  Std Deviation:    16,963.07
  Range:            161,978.06
  Min:              -184,081.68
  Max:              -22,103.61
  VaR (5%):         -100,315.43
  Expected SF (5%): -111,649.38


The hedged portfolio performed better than the unhedged portfolio in every metric. So incorporating options has reduced risk - no matter how minimial.

<h3><u>Visualisation of the two portfolios' metrics</u></h3>

In [61]:
fig = make_subplots(rows=1, cols=2,
                    shared_yaxes=True,
                    subplot_titles=('Unhedged Portfolio Final Values', 'Hedged Portfolio Final Values'))

fig.add_trace(
    go.Histogram(
        x=unhedged_portfolio_final_values,
        name='Unhedged Portfolio',
        marker_color='#636EFA',
        opacity=0.7,
        hovertemplate='Range: %{x}<br>Count: %{y}<extra></extra>'
    ),
    row=1, col=1
)

fig.add_vline(x=mean_unhedged, line_dash="dash", line_color="red",
              annotation_text=f"Mean: {mean_unhedged:,.2f}", annotation_position="top right",
              row=1, col=1)
fig.add_vline(x=var_unhedged_5_percent, line_dash="dot", line_color="orange",
              annotation_text=f"VaR (5%): {var_unhedged_5_percent:,.2f}", annotation_position="bottom right",
              row=1, col=1)

fig.add_trace(
    go.Histogram(
        x=hedged_portfolio_final_values,
        name='Hedged Portfolio',
        marker_color='#EF553B',
        opacity=0.7,
        hovertemplate='Range: %{x}<br>Count: %{y}<extra></extra>'
    ),
    row=1, col=2
)

fig.add_vline(x=mean_hedged, line_dash="dash", line_color="red",
              annotation_text=f"Mean: {mean_hedged:,.2f}", annotation_position="top right",
              row=1, col=2)
fig.add_vline(x=var_hedged_5_percent, line_dash="dot", line_color="orange",
              annotation_text=f"VaR (5%): {var_hedged_5_percent:,.2f}", annotation_position="bottom right",
              row=1, col=2)

fig.update_layout(
    title_text='Distribution of Portfolio Final Values: Unhedged vs. Hedged',
    xaxis_title_text='Final Portfolio Value',
    yaxis_title_text='Frequency',
    bargap=0.01,
    hovermode='x unified'
)

fig.show()

<h3><u>Tangency Portfolio against Real time data (JSE All-Share Index)</u></h3>

The data is from 1 January 2025 until 7 February 2026 - the day of writing of this project.

In [62]:
jasi['Date'] = pd.to_datetime(jasi['Date'], format='%Y/%m/%d')
print("Date column converted to datetime.")
start_date = '2025-01-01'
end_date = '2026-02-05'
jasi_filtered = jasi[(jasi['Date'] >= start_date) & (jasi['Date'] <= end_date)].copy()

jasi_filtered['Change %'] = jasi_filtered['Change %'].str.replace('%', '').astype(float) / 100
num_jse_trading_days = len(jasi_filtered.index)

simulated_returns_limited = {}

for ticker in portfolio_tickers:
    paths_limited = simulated_paths[ticker][:num_jse_trading_days + 1, :]

    daily_returns = pd.DataFrame(paths_limited).pct_change().iloc[1:, :]
    simulated_returns_limited[ticker] = daily_returns

simulated_portfolio_daily_returns = np.zeros((num_jse_trading_days, num_simulations))

for i in range(num_simulations):
    current_sim_returns_df = pd.DataFrame({
        ticker: simulated_returns_limited[ticker].iloc[:, i]
        for ticker in portfolio_tickers
    })

    portfolio_daily_return_for_sim = (current_sim_returns_df * tangency_portfolio_weights).sum(axis=1)

    simulated_portfolio_daily_returns[:, i] = portfolio_daily_return_for_sim

cumulative_returns_per_simulation = (1 + simulated_portfolio_daily_returns).cumprod(axis=0) - 1

mean_simulated_cumulative_returns = cumulative_returns_per_simulation.mean(axis=1)

jse_cumulative_filtered = (1 + jasi_filtered['Change %']).cumprod() - 1

simulated_dates = jasi_filtered['Date'].iloc[:num_jse_trading_days]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=simulated_dates,
    y=mean_simulated_cumulative_returns,
    mode='lines',
    name='Simulated Tangency Portfolio: Mean Cumulative Return'
))

fig.add_trace(go.Scatter(
    x=simulated_dates,
    y=jse_cumulative_filtered,
    mode='lines',
    name='JSE All Share Index: Cumulative Return'
))

fig.update_layout(
    title='Cumulative Returns: Simulated Tangency Portfolio vs. JSE All Share Index',
    xaxis_title='Date',
    yaxis_title='Cumulative Return',
    hovermode='x unified'
)

fig.show()

Date column converted to datetime.


# Final Answer

<u>Portfolio Performance Relative to Benchmark:</u>
<br>
Despite optimising for the highest Sharpe ratio, which remained negative, the simulated tangency portfolio experienced a negative mean cumulative return of $\approx$48.05%. In contrast, the JSE All-Share Index showed a positive cumulative return of approximately 2.37% over the same period. This indicates that the optimised portfolio - highly leveraged, with negative expected return - underperformed the market benchmark.
<br>
<br>

<u>Effectiveness of Hedging:</u>
<br>
The incorporation of derivatives (protective puts for long positions and protective calls for short positions) successfully reduced the portfolio's overall risk, albeit minimally. Key improvements in risk metrics were observed:
*   **Final Value**: The average final value of the portfolio increased from approximately -70,586.66 (unhedged) to -69,450.67 (hedged), implying a slight reduction in expected losses.
*   **Standard Deviation (Volatility)**: The standard deviation decreased from approximately 17,848.53 (unhedged) to 16,552.36 (hedged).
*   **Value at Risk (VaR) at 5%**: The 5% VaR improved, moving from a potential loss of -103,102.34 (unhedged) to -99,529.39 (hedged).
*   **Expected Shortfall (ES) at 5%**: The Expected Shortfall at 5% improved from -114,408.27 (unhedged) to -109,947.21 (hedged).

The metrics prove that hedging provided a level of risk mitigation, making the portfolio marginally less risky than the unhedged portfolio.
<br>

<u>Limitations:</u>
<br>
The analysis has a number of limitations:
1.  **Negative Expected Return**: The tangency portfolio itself yielded a negative expected daily return (-0.027431) and a high volatility (0.217221). This fundamentally makes it an undesirable (unprofitable) portfolio - again, this is a direct consequence of my method used to choose these tickers - purely out of popularity and not financial analysis.
2.  **High Leverage**: The tangency weights suggest an extremely leveraged strategy by the model, implying a disproportionately high level of borrowing and shorting. Such leverage magnifies both gains and losses, making the portfolio highly risky.
3.  **Asset Selection Basis**: The initial selection of equities was based firstly on popularity and then the highest Sharpe Ratios, rather than a deeper fundamental or technical analysis, which led to a portfolio with negative expected returns and significant short positions in assets that did not decline sufficiently to generate profit.

This analysis relies on historical mean returns and covariances, which are subject to estimation error and instability over time. Future projects will incorporate <i>better</i> optimisation techniques, Bayesian shrinkage estimators, or factor-based return models - learned in my honours degree - to mitigate sensitivity to noisy inputs.